In [ ]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    accounts = data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




In [ ]:
import pandas as pd 

from scripts.player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
product_A = game_master.construct_instance("inventory","product_A").get("ID")
bilding_B = game_master.construct_instance("building", "building_B", address="東京都品川区荏原3-7-13").get("ID")
asset_list = list(game_master.asset_registry.keys())
player1.redister_product(product_A)
player1.aquire_building(bilding_B, 2000)

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)

player1.sale_product(product_A, 80)

end_1 = ledger1.execute_settlement()
ledger1.display_transaction_history()

player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)

player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A)

end_2 = ledger1.execute_settlement()

ledger1.display_transaction_history()
# ledger1.display_trial_balance(end_1)
# ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")


In [ ]:
ledger1.display_financial_statements(end_1)

In [ ]:
inventory_c = game_master.construct_instance("inventory", "inventory_c")
type(inventory_c.get("class"))


In [ ]:
import pandas as pd 

from scripts.player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager

building_B = game_master.construct_instance("building")

In [ ]:
from scripts.player import *
 # ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーの初期化・登録
player1 = Player(name="Player1", game_master=game_master, initial_cash= 2000000)
player2 = Player(name="Player2", game_master=game_master, initial_cash= 2000000)
game_master.players.append(player1)
game_master.players.append(player2)

# 資産の生成と登録
building_1 = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding_1",
    value=1000000,
    address="東京都新宿区早稲田鶴巻町"
)["ID"]
building_2 = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding_2",
    value=1500000,
    address="東京都品川区荏原"
)["ID"]
product_A = game_master.construct_instance(
    asset_type="inventory",
    name="product_A"
)["ID"]
product_B = game_master.construct_instance(
    asset_type="inventory",
    name="product_B"
)["ID"]

# プレイヤーに資産を取得させる
player1.aquire_building(asset_id=building_1, value=1000000)
player2.aquire_building(asset_id=building_2, value=1500000)

# プレイヤーに商品を登録させる
player1.redister_product(product_id=product_A)
player2.redister_product(product_id=product_B)

# 商品を仕入れ、売り上げる
player1.purchase_product(product_id=product_A, quantity= 500, price= 120)
player2.purchase_product(product_id=product_B, quantity= 600, price= 80)

player1.sale_product(product_id=product_A, quantity= 320, sales_price= 300)
player2.sale_product(product_id=product_B, quantity= 500, sales_price= 210)

player1.sale_product(product_id=product_A, quantity= 150, sales_price =250)
player2.sale_product(product_id=product_B, quantity= 80) # 売価の更新なし

# ゲーム内時間を90日進める
print("\n=== 時間を365日進めます ===")
game_master.advance_time(days=365)


In [ ]:
# 資産の状態を表示
print("\n=== 資産の状態 ===")
for player in game_master.players:
    for asset_info in player.portfolio:
        asset_obj = game_master.get_asset_by_id(asset_info["ID"])
        if isinstance(asset_obj, asset.Tangible):
            print(f"プレイヤー名: {player.name}, 資産名: {asset_obj.name}, 帳簿価額: {int(asset_obj.value)}, 累計減価償却額: {int(asset_obj.accumulated_depreciation)}")
        if isinstance(asset_obj, asset.Inventory):
            print(f"プレイヤー名: {player.name}, 資産名: {asset_obj.name}, 帳簿価額: {int(asset_obj.value)}")
        

In [ ]:
# 勘定元帳の記録を確認
print("\n=== プレイヤー1の勘定元帳 ===")
player1.ledger_manager.display_transaction_history()

In [ ]:
# 勘定元帳の記録を確認
print("\n=== プレイヤー2の勘定元帳 ===")
player2.ledger_manager.display_transaction_history()

In [ ]:
end_1 = [player1.ends[0],player2.ends[0]]
player1.ledger_manager.display_financial_statements(end_1[0])

In [ ]:
player2.ledger_manager.display_financial_statements(end_1[1])

In [ ]:
from scripts.asset import *
b = Building(name="Test Building", value=100000, owner=None, address="Test Address")
print(isinstance(b, Tangible))  # True を期待

In [ ]:
import scripts.asset
print(asset_obj.__file__)


In [ ]:
from scripts.player import *

# ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player1 = Player(name="Player1", game_master=game_master)
game_master.players.append(player1)

# 資産の生成と登録
building_asset = game_master.construct_instance(
    asset_type="building",
    name="Office Building",
    value=1000000,
    address="Downtown City"
)

# プレイヤーに建物資産を取得させる
player1.aquire_building(asset_id=building_asset["ID"], value=1000000)

# ゲーム内時間を進める
print("\n=== 90日進行 ===")
game_master.advance_time(days=90)

# 資産の確認
print("\n=== プレイヤーのポートフォリオ ===")
for asset_info in player1.portfolio:
    asset_obj = asset_info.get("instance")
    print(f"資産名: {asset_obj.name}, 帳簿価額: {int(asset_obj.value)}, 累計減価償却額: {int(asset_obj.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player1.ledger_manager.display_transaction_history()